In [103]:
import pydocparser
import requests
from bs4 import BeautifulSoup
import webbrowser
import re

In [104]:

api_key = 'a97eb5d696857fea8bfa9e57fb9994017a3bb7da'
parser_id = 'paper_parser'

parser = pydocparser.Parser()
parser.login(api_key)
result = parser.ping()
print(result)

pong


In [105]:
id = parser.upload_file_by_path("data_samples/electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf", "paper_parser") 

In [106]:
data = parser.get_one_result("paper_parser", id)
data

[{'id': 'c6e924b6ad0cadbaecc66623c208609d',
  'document_id': '58ad73d3ccadac478dd59d2e607039f5',
  'remote_id': '',
  'file_name': 'electricity-theft-detection-using-machine-learning-IJERTCONV10IS04024.pdf',
  'media_link': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk',
  'media_link_original': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/original',
  'media_link_data': 'https://api.docparser.com/v1/document/media/lin7He2PPDPdAoIHbLLl3i3efaZVYkEKiQyu5qVX1Ah3alakMyntbBQaYV03A5jRTBjON-ogp_eW5dqfJw8rvR5rh-dkdqzHAsD8KyldkEk/data',
  'page_count': 4,
  'uploaded_at': '2024-01-24T15:50:35+00:00',
  'processed_at': '2024-01-24T15:50:36+00:00',
  'uploaded_at_utc': '2024-01-24T15:50:35+00:00',
  'uploaded_at_user': '2024-01-24T07:50:35+00:00',
  'processed_at_utc': '2024-01-24T15:5

In [107]:
keywords = data[0]['keywords']
references = data[0]['references']
# abstract = data[0]['abstract']

In [108]:
def convert_to_file(text, file_path):
    
    with open(file_path, 'w') as file:
        file.write(text)

    return file_path

def convert_to_plaintext(input_string):
    # Use regular expression to remove non-alphabetic characters
    clean_string = re.sub(r'[^a-zA-Z ]', '', input_string)
    return clean_string

In [109]:
convert_to_file(references, 'extracted_text/abc.txt')

'extracted_text/abc.txt'

In [110]:
def extract_words_in_quotes(input_string):
    clean = re.sub(r'\n', ' ', input_string)
    pattern = r'[“"]([^”"]*)[”"]'
    matches = re.findall(pattern, clean)
    
    return matches

In [111]:
names = extract_words_in_quotes(references)
names[1]

'Electricity theft detection in AMI using customers consumption patterns,'

Scraping

In [112]:
def get_abstract_from_paper_url(paper_page_url, headers):
    response = requests.get(paper_page_url, headers=headers)

    if response.status_code == 200:
        paper_soup = BeautifulSoup(response.text, 'html.parser')

        abstract_element = paper_soup.find('div', {'class': 'nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-800 research-detail-middle-section__abstract'})
        
        if abstract_element:
            abstract = abstract_element.get_text(strip=True)
            return abstract
        else:
            return "Abstract not found."
    else:
        return f"Failed to retrieve paper page: {response.status_code}"

In [123]:
def open_paper_link_by_name(paper_name):
    base_url = "https://www.researchgate.net/search/publication?q="
    search_url = base_url + paper_name.replace(" ", "+")
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }


    # Make a request to the ResearchGate search page
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all search results
        result_links = soup.find_all('a', {'class':  ['nova-legacy-e-link', 'nova-legacy-e-link--color-inherit', 'nova-legacy-e-link--theme-bare']})

        if result_links:
            print("Resutls found")

            for link in result_links:
                result_name = link.get_text(strip=True)
                # print("search: ",result_name)

                result_name = convert_to_plaintext(result_name)
                # print("plain_name:" ,result_name)

                # Check if the result name matches the given paper name
                if paper_name.lower() in result_name.lower():
                    paper_page_url = "https://www.researchgate.net/" + link['href']
                    abstract = get_abstract_from_paper_url(paper_page_url, headers=headers)
                    print("full link: ",paper_page_url)
                    print("abstact: ",abstract)

                    return result_name, paper_page_url, abstract


            print(f"No exact match found for '{paper_name}' in the search results.")    
            # return None, None, None
        else:
            return f"No results found for '{paper_name}'."
    else:
        return f"Failed to perform the search: {response.status_code}"

In [114]:
names[3]

'Scikit-learn: Machine learning in Python,'

In [124]:
paper_name_to_search = convert_to_plaintext(names[3])
result = open_paper_link_by_name(paper_name_to_search)


Resutls found
No exact match found for 'Scikitlearn Machine learning in Python' in the search results.


In [116]:
print(result)

None


In [122]:
if result:
    name, link, abstract1 = result[:3]
    print()
else:
    print("No result found")


No result found


NameError: name 'abstract1' is not defined